In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from tensorflow.keras import datasets,layers,models
from tensorflow.keras.utils import to_categorical
from keras import optimizers
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Conv2D, AveragePooling2D,MaxPooling2D, Dropout, BatchNormalization

%matplotlib inline

In [ ]:
cifar10 = tf.keras.datasets.cifar10
(X_train, Y_train), (X_test, Y_test) = cifar10.load_data()

In [ ]:
#checking the shape
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

In [ ]:
#converting values to float and normalizing
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train=X_train/255
X_test=X_test/255

In [ ]:
#encoding
Y_train = to_categorical(Y_train, 10)
Y_test = to_categorical(Y_test, 10)

In [ ]:
Input_shape = X_train.shape[1:]
Input_shape

In [ ]:
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
#visualising samples
plt.figure(figsize = (10,10))
for i in range(70):
  plt.subplot(10,10,1+i)
  plt.axis('off')
  plt.imshow(X_train[i], cmap = 'gray')

In [ ]:
cnn_model = models.Sequential()
cnn_model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu', input_shape = Input_shape))
cnn_model.add(Conv2D(filters = 64, kernel_size = (3,3), activation = 'relu'))
cnn_model.add(MaxPooling2D(2,2))
cnn_model.add(Dropout(0.4))


cnn_model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu'))
cnn_model.add(Conv2D(filters = 128, kernel_size = (3,3), activation = 'relu'))
cnn_model.add(AveragePooling2D(2,2))
cnn_model.add(Dropout(0.4))

cnn_model.add(Flatten())

cnn_model.add(Dense(units = 1024, activation = 'relu'))

cnn_model.add(Dense(units = 1024, activation = 'relu'))

cnn_model.add(Dense(units = 10, activation = 'softmax'))

In [ ]:
opt = optimizers.Adam(learning_rate = 0.001)
cnn_model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])

In [ ]:
cnn_model.summary()

In [ ]:
# fitting the train data into the model
history = cnn_model.fit(X_train, Y_train, batch_size = 32, epochs = 10, shuffle = True,validation_data=(X_test,Y_test))

In [ ]:
# Evaluating the model performance
evaluation = cnn_model.evaluate(X_test, Y_test)
print('Test Accuracy: {}'.format(evaluation[1]))

In [ ]:
predicted_classes = cnn_model.predict_classes(X_test) 
predicted_classes

In [ ]:
Y_test = Y_test.argmax(1)
Y_test

In [ ]:
# plotting Confusion Matrix
from sklearn.metrics import confusion_matrix
import seaborn as sns

cm = confusion_matrix(Y_test, predicted_classes)
cm
plt.figure(figsize = (10, 10))
sns.heatmap(cm, annot = True)

In [ ]:
# generating more data using the existing data
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
                            #width_shift_range = 0.1,
                            horizontal_flip = True,
                             )
datagen.fit(X_train)

In [ ]:
# retraining the model
cnn_model.fit_generator(datagen.flow(X_train, Y_train, batch_size = 32), epochs = 5)

In [ ]:
# encoding the Predictor variable
Y_test = to_categorical(Y_test, 10)

In [ ]:
# printing the accuracy
score = cnn_model.evaluate(X_test, Y_test)
print('Test accuracy', score[0])